In [ ]:
#call library

import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, StratifiedKFold, KFold

from keras.wrappers.scikit_learn import KerasClassifier




from preprocesslib import preprocessEach, makeY, splitData, splitDataCrossVal


In [ ]:
xGrowths = preprocessEach('growth')
xValues = preprocessEach('value')
xOverlaps = preprocessEach('overlap')

In [ ]:
#bootstrap
n_samples = len(xValues)-len(xGrowths)
xGrowths_b = xGrowths[np.random.choice(len(xGrowths), size=n_samples, replace=True)]
xGrowths = np.concatenate((xGrowths, xGrowths_b))


In [ ]:
n_samples = len(xValues)-len(xOverlaps)
xOverlaps_b = xOverlaps[np.random.choice(len(xOverlaps), size=n_samples, replace=True)]
xOverlaps = np.concatenate((xOverlaps, xOverlaps_b))

In [ ]:
#make y values
yGrowths = makeY('growth', len(xGrowths))
yValues = makeY('value', len(xValues))
yOverlaps = makeY('overlap', len(xOverlaps))

In [ ]:
x = np.concatenate((xGrowths, xValues, xOverlaps), axis=0)
y = np.concatenate((yGrowths, yValues, yOverlaps), axis=0)

In [ ]:
#minmax scaling by element
scaler = MinMaxScaler()
for i in range(6):
    x[:,:,i] = scaler.fit_transform(x[:,:,i])

In [ ]:
LAYERS=1
def make_gru(u,d):
    model = Sequential()
    model.add(GRU(u, input_shape=(1248, 6)))
    # model.add(GRU(u, input_shape=(1248, 6), return_sequences=True))
    model.add(Dropout(d))
    # model.add(GRU(int(u/2)))
    # model.add(Dropout(d))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [ ]:
units =  [8]
dropouts = [0]
epochs = [2000]
batchs = [64]
p = 100
# gru_hist = pd.DataFrame(columns=['epochs', 'units', 'dropouts', 'layers','batchs','patience', 'acc', 'wholeacc'])
gru_hist = pd.read_csv('./GRU/GRU_result_2.csv')
kf = KFold(3, shuffle=True)

for b in batchs:
    for e in epochs:
        for u in units:
            for d in dropouts:
                print("======================================================================================")
                print(f"units: {u}, dropout: {d}, epoch: {e}")
                accuracy_cross_val = []
                for id, (train_index, test_index) in enumerate(kf.split(x)):
                    x_train, x_test = x[train_index], x[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    model = make_gru(u,d)
                    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=p)
                    # reduce_lr = ReduceLROnPlateau()
                    mc = ModelCheckpoint(f'./GRU/GRU__units_{u} dropout_{d} epoch_{e}_b_{b}_model.h5', monitor='val_acc', mode='max', save_best_only=True, )
                    history=model.fit(x_train, y_train, epochs=e, batch_size=b, validation_split=0.3, callbacks=[es, mc])
                    model=load_model(f'./GRU/GRU__units_{u} dropout_{d} epoch_{e}_b_{b}_model.h5')
                    loss, accuracy = model.evaluate(x_test, y_test)
                    accuracy_cross_val.append(accuracy)
                    # history=model.fit(x_train, y_train, epochs=e, batch_size=b, validation_data = (x_test, y_test),callbacks= [es,mc])
                    # accuracy_cross_val.append(mc.best)
                # bestid = np.argmax(accuracy_cross_val)
                epochs = range(1, len(history.history['acc']) + 1)
                plt.figure(figsize=(10, 6))
                plt.plot(epochs, history.history['loss'])
                plt.plot(epochs, history.history['val_loss'])
                plt.title('model loss')
                plt.ylabel('loss')
                plt.xlabel('epoch')
                plt.legend(['train', 'val'], loc='upper left')
                plt.savefig(f"GRU/1_units_{u} dropout_{d} epoch_{e}_cv{len(accuracy_cross_val)}_layer{LAYERS}_b_{b}_p_{p}.png")
                plt.close()

                plt.figure(figsize=(10, 6))
                plt.plot(epochs, history.history['acc'])
                plt.plot(epochs, history.history['val_acc'])
                plt.title('model acc')
                plt.ylabel('acc')
                plt.xlabel('epoch')
                plt.legend(['train', 'val'], loc='upper left')
                plt.savefig(f"GRU/2_units_{u} dropout_{d} epoch_{e}_cv{len(accuracy_cross_val)}_layer{LAYERS}_b_{b}_p_{p}.png")
                plt.close()
                gru_hist.loc[gru_hist.shape[0]] = [e,u,d,LAYERS, b,p,np.mean(accuracy_cross_val), accuracy_cross_val]

In [ ]:
gru_hist.to_csv('./GRU/GRU_result_2.csv')

In [ ]:
p = 75


In [ ]:
gru_hist = pd.DataFrame(columns=['epochs', 'units', 'dropouts', 'layers','batchs','patience', 'acc', 'wholeacc'])
gru_hist.loc[gru_hist.shape[0]] = [e,u,d,LAYERS, b,p,np.mean(accuracy_cross_val), accuracy_cross_val]
gru_hist.to_csv('./GRU/GRU_result_2.csv')
